In [ ]:
%%capture
import os
import site
os.sys.path.insert(0, '/home/schirrmr/code/reversible/reversible2/')
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/braindecode/')
os.sys.path.insert(0, '/home/schirrmr/code/explaining/reversible//')
%cd /home/schirrmr/


%load_ext autoreload
%autoreload 2
import numpy as np
import logging
log = logging.getLogger()
log.setLevel('INFO')
import sys
logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
%matplotlib inline
%config InlineBackend.figure_format = 'png'
matplotlib.rcParams['figure.figsize'] = (12.0, 1.0)
matplotlib.rcParams['font.size'] = 14
import seaborn
seaborn.set_style('darkgrid')

from reversible.sliced import sliced_from_samples

from numpy.random import RandomState

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
import copy
import math

import itertools
from reversible.plot import create_bw_image
import torch as th
from braindecode.torch_ext.util import np_to_var, var_to_np
from reversible.revnet import ResidualBlock, invert, SubsampleSplitter, ViewAs, ReversibleBlockOld
from spectral_norm import spectral_norm
from conv_spectral_norm import conv_spectral_norm

def display_text(text, fontsize=18):
    fig = plt.figure(figsize=(12,0.1))
    plt.title(text, fontsize=fontsize)
    plt.axis('off')
    display(fig)
    plt.close(fig)

In [ ]:
from braindecode.datasets.bbci import BBCIDataset
from braindecode.mne_ext.signalproc import mne_apply
# we loaded all sensors to always get same cleaning results independent of sensor selection
# There is an inbuilt heuristic that tries to use only EEG channels and that definitely
# works for datasets in our paper
#train_loader = BBCIDataset('/data/schirrmr/schirrmr/HGD-public/reduced/train/13.mat')
#test_loader = BBCIDataset('/data/schirrmr/schirrmr/HGD-public/reduced/test/13.mat')
start_cnt = BBCIDataset('/data/schirrmr/schirrmr/HGD-public/reduced/train/4.mat',).load()
start_cnt = start_cnt.drop_channels(['STI 014'])
def car(a):
    return a - np.mean(a, keepdims=True, axis=0)

start_cnt = mne_apply(
    car, start_cnt)

start_cnt = start_cnt.reorder_channels(['C3', 'C4'])

In [ ]:
from collections import OrderedDict
from braindecode.datautil.trial_segment import create_signal_target_from_raw_mne

marker_def = OrderedDict([('Right Hand', [1]), ('Left Hand', [2],),
                         ('Rest', [3]), ('Feet', [4])])
ival = [500,1500]
from braindecode.mne_ext.signalproc import mne_apply, resample_cnt
from braindecode.datautil.signalproc import exponential_running_standardize, bandpass_cnt

log.info("Resampling train...")
cnt = resample_cnt(start_cnt, 250.0)
log.info("Standardizing train...")
cnt = mne_apply(lambda a: exponential_running_standardize(a.T ,factor_new=1e-3, init_block_size=1000, eps=1e-4).T,
                     cnt)
cnt = resample_cnt(cnt, 32.0)
cnt = resample_cnt(cnt, 64.0)
#cnt = mne_apply(
#    lambda a: bandpass_cnt(a, 0, 2, cnt.info['sfreq'],
#                           filt_order=10,
#                           axis=1), cnt)

train_set = create_signal_target_from_raw_mne(cnt, marker_def, ival)

In [ ]:
x_right = train_set.X[train_set.y == 0]

x_rest = train_set.X[train_set.y == 2]

inputs_a = np_to_var(x_right[:160,0:1,:,None], dtype=np.float32)

inputs_b = np_to_var(x_rest[:160,0:1,:,None], dtype=np.float32)
inputs = [inputs_a, inputs_b]

In [ ]:
fig = plt.figure(figsize=(8,4))
for i_class in range(len(inputs)):
    ins = var_to_np(inputs[i_class].squeeze())
    bps = np.abs(np.fft.rfft(ins.squeeze()))
    plt.plot(np.fft.rfftfreq(ins.squeeze().shape[1], d=1/ins.squeeze().shape[1]), np.median(bps, axis=0))

    
plt.title("Spectrum")
plt.xlabel('Frequency [Hz]')

plt.ylabel('Amplitude')
plt.legend(['Real Right', 'Fake Right', 'Real Rest', 'Fake Rest'])
display(fig)
plt.close(fig)

In [ ]:
def rev_block(n_c, n_i_c):
     return ReversibleBlockOld(
        nn.Sequential(
        nn.Conv2d(n_c // 2, n_i_c,(3,1), stride=1, padding=(1,0),bias=True),
        nn.ReLU(),
            nn.Conv2d(n_i_c, n_c // 2,(3,1), stride=1, padding=(1,0),bias=True)),
         
        nn.Sequential(
        nn.Conv2d(n_c // 2, n_i_c,(3,1), stride=1, padding=(1,0),bias=True),
        nn.ReLU(),
            nn.Conv2d(n_i_c, n_c // 2,(3,1), stride=1, padding=(1,0),bias=True))
    )
def dense_rev_block(n_c, n_i_c):
     return ReversibleBlockOld(
        nn.Sequential(
            nn.Linear(n_c // 2, n_i_c, bias=True),
            nn.ReLU(),
            nn.Linear(n_i_c, n_c // 2,bias=True)),
        nn.Sequential(
            nn.Linear(n_c // 2, n_i_c, bias=True),
            nn.ReLU(),
            nn.Linear(n_i_c, n_c // 2, bias=True))
    )
    
def res_block(n_c, n_i_c):
     return ResidualBlock(
        nn.Sequential(
        nn.Conv2d(n_c, n_i_c, (3,1), stride=1, padding=(1,0),bias=True),
        nn.ReLU(),
            nn.Conv2d(n_i_c, n_c, (3,1), stride=1, padding=(1,0),bias=True)),
    )

In [ ]:
from rfft import RFFT, Interleave

In [ ]:
from discriminator import ProjectionDiscriminator
from reversible.revnet import SubsampleSplitter, ViewAs
from reversible.util import set_random_seeds
from reversible.revnet import init_model_params
from torch.nn import ConstantPad2d
import torch as th
from conv_spectral_norm import conv_spectral_norm
from disttransform import DistTransformResNet


set_random_seeds(2019011641, True)
feature_model = nn.Sequential(
    ViewAs((-1,1,64,1), (-1,64)),
    RFFT(),
)



from braindecode.torch_ext.util import set_random_seeds
adv_model = nn.Sequential(
    nn.Conv2d(1,16, (3,1), stride=1, padding=(1,0),bias=True),
    res_block(16,32),
    res_block(16,32),
    nn.AvgPool2d((2,1)),
    res_block(16,32),
    res_block(16,32),
    nn.AvgPool2d((2,1)),
    res_block(16,32),
    res_block(16,32),
    nn.AvgPool2d((2,1)),
    res_block(16,32),
    res_block(16,32),
    nn.AvgPool2d((2,1)),
    res_block(16,32),
    res_block(16,32),
    nn.AvgPool2d((2,1)),
    res_block(16,32),
    res_block(16,32),
    ViewAs((-1,16,2,1), (-1,32)),
    )


adv_model = ProjectionDiscriminator(adv_model,32,2,)




from reversible.training import hard_init_std_mean
n_dims = inputs[0].shape[2]
n_clusters = 2
means_per_cluster = [th.autograd.Variable(th.ones(n_dims), requires_grad=True)
                     for _ in range(n_clusters)]
# keep in mind this is in log domain so 0 is std 1
stds_per_cluster = [th.autograd.Variable(th.zeros(n_dims), requires_grad=True)
                    for _ in range(n_clusters)]

for i_class in range(n_clusters):
    this_outs = feature_model(inputs[i_class])
    means_per_cluster[i_class].data = th.mean(this_outs, dim=0).view(-1).data
    stds_per_cluster[i_class].data = th.log(th.std(this_outs, dim=0),).view(-1).data



from copy import deepcopy
optimizer = th.optim.Adam(
                          [
    {'params': list(feature_model.parameters()),
    'lr': 1e-3,
    'weight_decay': 0},], betas=(0,0.9))

optim_dist = th.optim.Adam(
                          [
    {'params': means_per_cluster + stds_per_cluster,
    'lr': 1e-3,
    'weight_decay': 0},], betas=(0,0.9))

optim_adv = th.optim.Adam([{
    'params': adv_model.parameters(),
    'lr': 1e-3, 'weight_decay': 0.00}],#lr 0.0004
                         betas=(0,0.9))



In [ ]:

from reversible.gaussian import get_gauss_samples
from reversible.uniform import get_uniform_samples

from reversible.gaussian import get_gauss_samples
from reversible.uniform import get_uniform_samples
from reversible.revnet import invert 
import pandas as pd
from gradient_penalty import gradient_penalty
import time


df = pd.DataFrame()
g_loss = np_to_var([np.nan],dtype=np.float32)
g_grad = np.nan
d_loss = np_to_var([np.nan],dtype=np.float32)
d_grad = np.nan
gradient_loss = np_to_var([np.nan],dtype=np.float32)

In [ ]:
def invert_hierarchical(features):
    return invert(feature_model, features)


def get_samples(n_samples, i_class):
    mean = means_per_cluster[i_class]
    std = th.exp(stds_per_cluster[i_class])
    # let's create a mask for the std for now
    samples = get_gauss_samples(n_samples, mean, std, truncate_to=3)
    return samples

In [ ]:
n_epochs = 2001
rng = RandomState(349384)
for i_epoch in range(n_epochs):
    start_time = time.time()
    optim_adv.zero_grad()
    optimizer.zero_grad()
    optim_dist.zero_grad()
    for i_class in range(len(inputs)):
        this_inputs = inputs[i_class] 
        n_samples = len(this_inputs)
        samples = get_samples(n_samples, i_class)
        inverted = invert_hierarchical(samples)
        y = np_to_var([i_class])
        score_fake = adv_model(inverted, y)
        if (i_epoch % 10) != 0:
            score_real = adv_model(this_inputs, y)
            gradient_loss = gradient_penalty(adv_model, this_inputs, inverted[:(len(this_inputs))], y)
            d_loss = -score_real.mean() + score_fake.mean() + gradient_loss * 10
            d_loss.backward()
            d_grad = np.mean([th.sum(p.grad **2).item() for p in adv_model.parameters()])
        else:
            g_loss = -th.mean(score_fake)
            g_loss.backward()
            g_grad = np.mean([th.sum(p.grad **2).item() for p in itertools.chain(feature_model.parameters(),
                                                                     feature_model.parameters())])
    if (i_epoch % 10) != 0:
            optim_adv.step()
    else:
        optimizer.step()
        optim_dist.step()
    with th.no_grad():
        sample_wd_row = {}
        for i_class in range(len(inputs)):
            this_inputs = inputs[i_class]
            n_samples = len(this_inputs)
            samples = get_samples(n_samples, i_class)
            inverted = invert_hierarchical(samples)
            in_np = var_to_np(this_inputs).reshape(len(this_inputs), -1)
            fake_np = var_to_np(inverted).reshape(len(inverted), -1)
            import ot

            dist = np.sqrt(np.sum(np.square(in_np[:,None] - fake_np[None]), axis=2))
            match_matrix = ot.emd([],[], dist)
            cost = np.sum(dist * match_matrix)
            score_fake = adv_model(inverted, y)
            score_real = adv_model(inputs[i_class], y)
            wd_dist = th.mean(score_real) - th.mean(score_fake)
            sample_wd_row.update({
                'wd_dist_' + str(i_class): wd_dist.item() ,
                'sampled_wd' + str(i_class): cost,
                'wd_diff' + str(i_class): cost - wd_dist.item(),
            })
        end_time = time.time()
        epoch_row = {
        'd_loss': d_loss.item(),
        'g_loss': g_loss.item(),
        'grad_loss': gradient_loss.item(),
        'o_real': th.mean(score_real).item(),
        'o_fake': th.mean(score_fake).item(),
        'g_grad': g_grad,
        'd_grad': d_grad,
        'runtime': end_time -start_time,}
        epoch_row.update(sample_wd_row)
        df = df.append(epoch_row, ignore_index=True)
        if i_epoch % (max(1,n_epochs // 20)) == 0:
            display_text("Epoch {:d}".format(i_epoch))
            display(df.iloc[-5:])
        if i_epoch % (n_epochs // 20) == 0:
            print("stds\n", var_to_np(th.exp(th.stack(stds_per_cluster))))
            fig = plt.figure(figsize=(8,4))
            plt.plot(var_to_np(th.exp(th.stack(stds_per_cluster))).squeeze().T)
            plt.title("Standard deviation\nper dimension")
            display(fig)
            plt.close(fig)
            
            
            fig = plt.figure(figsize=(8,4))
            for i_class in range(len(inputs)):
                ins = var_to_np(inputs[i_class].squeeze())
                bps = np.abs(np.fft.rfft(ins.squeeze()))
                plt.plot(np.fft.rfftfreq(ins.squeeze().shape[1], d=1/ins.squeeze().shape[1]), np.median(bps, axis=0))

                n_samples = 5000
                samples = get_samples(n_samples, i_class)
                inverted = var_to_np(invert_hierarchical(samples).squeeze())
                bps = np.abs(np.fft.rfft(inverted.squeeze()))
                plt.plot(np.fft.rfftfreq(inverted.squeeze().shape[1], d=1/ins.squeeze().shape[1]), np.median(bps, axis=0),
                        color=seaborn.color_palette()[i_class], ls='--')
            plt.title("Spectrum")
            plt.xlabel('Frequency [Hz]')

            plt.ylabel('Amplitude')
            plt.legend(['Real Right', 'Fake Right', 'Real Rest', 'Fake Rest'])
            display(fig)
            plt.close(fig)
            
            for i_class in range(len(inputs)):
                fig = plt.figure(figsize=(5,5))
                mean = means_per_cluster[i_class]
                log_std = stds_per_cluster[i_class]
                std = th.exp(log_std)
                y = np_to_var([i_class])
                n_samples = 5000
                samples = get_samples(n_samples, i_class)
                inverted = var_to_np(invert_hierarchical(samples).squeeze())
                plt.plot(inverted.squeeze()[:,0], inverted.squeeze()[:,1],
                         ls='', marker='o', color=seaborn.color_palette()[i_class + 2], alpha=0.5, markersize=2)
                plt.plot(var_to_np(inputs[i_class].squeeze())[:,0], var_to_np(inputs[i_class].squeeze())[:,1],
                         ls='', marker='o', color=seaborn.color_palette()[i_class])

                display(fig)
                plt.close(fig)
                fig = plt.figure(figsize=(8,3))
                plt.plot(inverted[:1000].T, color=seaborn.color_palette()[0],lw=0.5);
                display(fig)
                plt.close(fig)
                
                i_dims = np.argsort(var_to_np(stds_per_cluster[0]))[::-1][:2]

                with th.no_grad():
                    mean = means_per_cluster[i_class]
                    std = th.exp(stds_per_cluster[i_class])
                    samples = get_samples(5000, i_class)
                    outs = feature_model(inputs[i_class])
                fig = plt.figure(figsize=(3,3))
                plt.plot(var_to_np(samples)[:,i_dims[0]].squeeze(),
                         var_to_np(samples)[:,i_dims[1]].squeeze(), marker='o', ls='')
                plt.plot(var_to_np(outs)[:,i_dims[0]].squeeze(),
                         var_to_np(outs)[:,i_dims[1]].squeeze(), marker='o', ls='')
                plt.legend(["Fake", "Real"])
                display(fig)
                plt.close(fig)
            i_dims = (np.argsort(np.max(var_to_np(th.stack(stds_per_cluster)), axis=0))[::-1][:4])
            for i_dim in i_dims:
                display_text("Dimension {:d}".format(i_dim))
                examples_per_class = []
                outs_per_class = []
                for i_class in range(2):
                    mean = means_per_cluster[i_class]
                    std = th.exp(stds_per_cluster[i_class])
                    i_f_vals = th.linspace((mean[i_dim] - 2 * std[i_dim]).item(),
                                           (mean[i_dim] +  2 *std[i_dim]).item(), 21)
                    examples = mean.repeat(len(i_f_vals), 1)
                    examples.data[:,i_dim] = i_f_vals.data
                    examples_per_class.append(examples)
                    outs_per_class.append(feature_model(inputs[i_class]))
                #display_text(["Right", "Rest"][i_class])
                fig, axes = plt.subplots(1,2, figsize=(6,3), sharex=True, sharey=True)
                for i_class in range(2):
                    from matplotlib import rcParams, cycler
                    cmap = plt.cm.coolwarm
                    N = len(examples)
                    examples = examples_per_class[i_class]
                    axes[i_class].plot(var_to_np(outs_per_class[i_class])[:,i_dim].squeeze(),
                                       var_to_np(outs_per_class[i_class])[:,i_dim].squeeze() * 0 - 0.01,
                                      ls='', marker='o', alpha=0.25, markersize=3,
                                      color=seaborn.color_palette()[i_class])
                    axes[i_class].scatter(var_to_np(examples)[:,i_dim].squeeze(),
                                          var_to_np(examples)[:,i_dim].squeeze() * 0,
                       c=cmap(np.linspace(0, 1, N)))
                    if i_class == 0:
                        axes[i_class].set_title("Latent space:")

                display(fig)
                plt.close(fig)
                with plt.rc_context({'axes.prop_cycle': cycler(color=cmap(np.linspace(0, 1, N)))}):
                    fig, axes = plt.subplots(1,2, figsize=(16,3), sharex=True, sharey=True)
                    for i_class in range(2):
                        inverted = invert_hierarchical(examples_per_class[i_class])
                        axes[i_class].plot(var_to_np(inverted).squeeze().T);
                    display(fig)
                    plt.close(fig)



In [ ]:
adv_model.adv_feature_model[0].weight